## OBJECT DETCTION USING OPENCV

### PART 1: Object Detection on the Test Images

In [ ]:
import cv2
import numpy as np
import os

input_images = ['dog_and_cat.jpg', 'apple.jpg']
os.makedirs('output', exist_ok=True)

for i, img_path in enumerate(input_images):
    img = cv2.imread(img_path)
    orig = img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10
    )

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (12, 12))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    img_area = img.shape[0] * img.shape[1]
    min_area = 0.01 * img_area  

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > min_area:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 255, 0), 4)

    cv2.imwrite(f'output/detected_image_{i+1}.jpg', orig)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


### PART 2: Background Removal on the Test Images

In [9]:
for i, img_path in enumerate(input_images):
    img = cv2.imread(img_path)

    lower = np.array([228, 228, 228], dtype=np.uint8)
    upper = np.array([255, 255, 255], dtype=np.uint8)

    mask = cv2.inRange(img, lower, upper)
    not_mask = cv2.bitwise_not(mask)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    not_mask = cv2.morphologyEx(not_mask, cv2.MORPH_OPEN, kernel)
    not_mask = cv2.morphologyEx(not_mask, cv2.MORPH_CLOSE, kernel)

    result = cv2.bitwise_and(img, img, mask=not_mask)

    white_background = np.full_like(img, 255)
    inv_mask = cv2.bitwise_not(not_mask)
    clean_background = cv2.bitwise_and(white_background, white_background, mask=inv_mask)

    final = cv2.add(result, clean_background)
    cv2.imwrite(f'output/segmented_object_{i+1}.png', final)


### PART 3: Background Removal from Webcam

In [ ]:
cap = cv2.VideoCapture(0)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
out = cv2.VideoWriter('output/realtime_background_removed.mp4', fourcc, 20.0, (frame_width, frame_height))

lower_black = np.array([0, 0, 0])
upper_black = np.array([180, 255, 60])
lower_white = np.array([0, 0, 145])
upper_white = np.array([180, 30, 255])
while True:
    _, frame = cap.read()

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask_black = cv2.inRange(hsv, lower_black, upper_black)
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    mask_bg = cv2.bitwise_or(mask_black, mask_white)
    mask_fg = cv2.bitwise_not(mask_bg) 

    mask_clean = cv2.morphologyEx(mask_fg, cv2.MORPH_OPEN, kernel, iterations=3)
    mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_CLOSE, kernel, iterations=3)
    mask_img = cv2.merge([mask_clean] * 3)

    foreground = cv2.bitwise_and(frame, mask_img)
    white_bg = np.full(frame.shape, 255, dtype=np.uint8)
    inv_mask = cv2.bitwise_not(mask_clean)
    inv_mask_img = cv2.merge([inv_mask]*3)
    background = cv2.bitwise_and(white_bg, inv_mask_img)

    output = cv2.add(foreground, background)
    out.write(output)  
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
